#### Sociology 128D: Mining Culture Through Text Data: Introduction to Social Data Science – Summer '22

# Notebook 12: Evaluating Topic Models

LDA is an unsupervised machine learning method, but it requires a lot of input from the researcher. Most obviously, the researcher has to choose the number of topics. LDA also requires [priors](https://en.wikipedia.org/wiki/Prior_probability) about the distribution of topics and the distribution of words. In this notebook we will explore how to select hyperparameters like the number of topics (*k*) by training many models with different values and comparing them.

Often when researchers go through this process, there may be a straightforward way to tell whether they are choosing better or worse values of a hyperparameter. Models intended to classify documents (e.g., as spam) can be evaluated to see how accurate they are. That is "supervised" machine learning. When we say LDA is "unsupervised" we mean that we cannot straightforwardly say we have a measure of how good a model is. LDA creates categories and asserts that documents are mixtures of them, which is very different from assigning each document to a discrete, pre-established category. Evaluating a topic model can thus be more complicated.

Numerous methods for evaluating statistical properties of topic models have been devised. We will consider three. Sometimes researchers will evaluate the quality of topic models on their own. We will also do a bit of that. In other cases, researchers ask other people to evaluate topics–these may be domain experts or survey participants, and methods may be informal or even [experimental](https://papers.nips.cc/paper/2009/file/f92586a25bb3145facd64ab20fd554ff-Paper.pdf).

### Setup

You will need to install `gensim` if you have not. You will also need to install `wordcloud` and `pyLDAvis` (e.g., <tt>conda install -c conda-forge wordcloud</tt>). Importing `pyLDAvis` in the final portion of the notebook may raise a few warnings. For purposes of this notebook, you can disregard them.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pickle
import seaborn as sns
import time
import warnings

from gensim.corpora.dictionary import Dictionary
from gensim.models.callbacks import CoherenceMetric, PerplexityMetric
from gensim.models.coherencemodel import CoherenceModel
from gensim.models.ldamodel import LdaModel
from gensim.models.ldamulticore import LdaMulticore
from gensim.models.phrases import Phrases
from wordcloud import WordCloud

warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
workers = max(1, os.cpu_count()-1) # needed if you implement LdaMulticore

os.makedirs("models/", exist_ok=True)

### Data

The dataset for this notebook (and Notebook 10) is a (fixed!) preprocessed version of the Reddit r/jobs corpus we have used before. You can find the file on Canvas (<tt>Files -> Data -> rjobs_preprocessed_v2.json</tt>).

This corpus comprises posts submitted to [r/jobs](https://www.reddit.com/r/jobs/) between the morning of January 1, 2020, and the morning of January 1, 2021. The <tt>text</tt> field is the original title combined with the original text of the body of the post (that is, the <tt>title</tt> and <tt>selftext</tt> fields in other files we have used). The <tt>preprocessed</tt> field is a version of the text that has had URLs removed, contractions expanded (e.g., "shan't" -> "shall not"), stop words and single characters removed, and ordinal numbers converted to text (e.g., "1st" -> "first"). Remaining tokens have been lemmatized.

In [ ]:
np.random.seed(5971) # from random.org

df = pd.read_json("data/rjobs_preprocessed_v3.json")

preprocessed = df.preprocessed.apply(str.split).tolist()
print(len(preprocessed))
print(preprocessed[0])

In [ ]:
df.head()

In [ ]:
df.shape

### Identifying *n*-grams

We will use the function we used in Notebook 10 to identify *n*-grams. You can see more details about this function in that notebook.

In [ ]:
def train_ngram_model(docs: list, min_count: int=5, inc_trigrams: bool=True) -> list:
    """Returns documents with n-grams joined by underscores"""
    docs = [doc for doc in docs if doc] # the "if doc" condition removes empty strings (docs with no words)
    bigram_model = Phrases(docs, min_count=min_count)
    ngrams = bigram_model[docs]
    ngrams = list(ngrams)
    if inc_trigrams:
        trigram_model = Phrases(ngrams, min_count=min_count)
        ngrams = trigram_model[ngrams]
        ngrams = list(ngrams)
    return ngrams

In [ ]:
ngrams = train_ngram_model(preprocessed, min_count=25)

In [ ]:
print(ngrams[753])

### Train and Test Sets

To speed up the notebook, we are going to use a subset of the documents. `subset = ngrams[:10000]` will take the first 10,000 documents of the corpus and assign them to our new list. Training the models on the full dataset takes about five hours on my laptop, whereas the subset takes about one hour. You can make changes like trying more or fewer values of *k*, trying different options for alpha, or tuning other hyperparemters. These may affect the number of documents you want to consider at first.

Two approaches we will use to evaluate topic models quanitatively are applied to new or "held out" data. In case you are unfamiliar with this practice, the basic idea is that when we train a model, it may learn weird, quirky things about the specific data points–in our case, documents–that it is trained on. In a machine learning context, we often use [cross-validation](https://en.wikipedia.org/wiki/Cross-validation_(statistics)) to account for this. This means training the model on one set of data points and then seeing how well it fits a new set of data points that it has not seen. If a model does poorly when tested using new data (e.g., to make predictions), we make changes. In this case, we are going to assess measures called perplexity and topic coherence using a test set.

We are going to use a basic 70%/30% split for the train and test sets, respectively. `np.random.shuffle()` shuffles the order of the documents in place, and we will then create <tt>train</tt> and <tt>test</tt> by using list slices.

In [ ]:
subset = ngrams[:10000]

np.random.shuffle(subset)
train = subset[:7000]
test = subset[7000:]

In [ ]:
len(train), len(test)

### Gensim Dictionary and Bag-of-Words Corpus

Now we will create our dictionary object using the `Dictionary` class, filter rare and excessively frequently words, and convert the documents to the `gensim` bag-of-words format using the `.doc2bow()` method. These steps are covered in more detail when they are introduced in [Notebook 11](https://soc128d.github.io/notebooks/).

In [ ]:
dictionary = Dictionary(train)
print(dictionary[0])
print(len(dictionary))

In [ ]:
dictionary.filter_extremes(no_below=5, no_above=0.99)
print(len(dictionary))

In [ ]:
train_corpus = [dictionary.doc2bow(doc) for doc in train]
test_corpus = [dictionary.doc2bow(doc) for doc in test]

In [ ]:
print(len(train_corpus), len(test_corpus))

Now let's save these resources in case we need to restart the notebook.

In [ ]:
dictionary.save("notebook12_dictionary.d")
pickle.dump(train_corpus, open("notebook12_train_corpus.p", "wb"))
pickle.dump(test_corpus, open("notebook12_test_corpus.p", "wb"))

In [ ]:
dictionary = Dictionary.load("notebook12_dictionary.d")
train_corpus = pickle.load(open("notebook12_train_corpus.p", "rb"))
test_corpus = pickle.load(open("notebook12_test_corpus.p", "rb"))

In [ ]:
print(len(dictionary), len(train_corpus), len(test_corpus))

### Logging Coherence and Perplexity During Training

Although our focus is necessarily on choosing hyperparameters–at least in the case of *k*–one point I want to continue to make is that it is often useful, if not necessary, to train certain classes of models for longer than expected and to carefully monitor their progress. In Notebook 10, we saw that we can train a topic model with a line of code. That does not necessarily mean this is a *good* model. Here we will draw on [an example from Megan Stodel](https://www.meganstodel.com/posts/callbacks/). I have tweaked her implementation of [`gensim.models.callbacks`](https://radimrehurek.com/gensim/models/callbacks.html), which focused on other issues. We are going to focus on *coherence* and *perplexity*.

[Coherence metrics](https://svn.aksw.org/papers/2015/WSDM_Topic_Evaluation/public.pdf) are intended to quantify topic quality. In practical terms, this generally means looking at the extent to which the top words for a topic co-occur. Perplexity is a measure of the quality of the model overall–not the topics–and relates to how "unexpected" the data are, given the model. We want to maximize coherence and minimize perplexity. `gensim.models.callbacks` allows us to log the coherence and perplexity after each time the model passes through the full dataset (the <tt>passes</tt> argument for the `LdaModel()` method, also referred to as epochs in general). We will train the models for multiple epochs to assess whether the models improve as they continue to train.

The <tt>cv_coherence_logger</tt> and <tt>perplexity_logger</tt> we create below will be used during the training process. These will use the *test data* to evaluate the models.

In [ ]:
cv_coherence_logger = CoherenceMetric(texts=test, logger="shell", coherence="c_v")
perplexity_logger = PerplexityMetric(corpus=test_corpus, logger="shell")

## Hyperparameter Tuning

A [hyperparameter](https://en.wikipedia.org/wiki/Hyperparameter_(machine_learning)) is a value (or starting value) used to influence how a model learns. For LDA, the most obvious choice we have to make is the number of topics, *k*, that we want the model to learn. There are other hyperparameters, however. We are also going to try different *priors* about the distribution of the topics (*alpha*) in line with some well-regarded [research](https://papers.nips.cc/paper/2009/file/0d0871f0806eae32d30983b62252da50-Paper.pdf), and we will leave the other major prior, (b)eta, alone for now.

The next cell will iterate through different numbers of topics and different alphas, train topic models using them, and save the results, including the coherence and perplexity scores. We can use these saved results to plot the coherence and perplexity for different values of the hyperparameters and make a decision about which model is best. We are setting this up in a particular way so that we can see that the models improve over time during the training process. We will try values of *k* from 2-5 as well as 10, 20, 30, 40, and 50. For alpha, we will try `gensim`'s default option ("symmetric"), the "auto" option (which learns an asymmetric prior), and 50/*k*. The creator of `gensim` blogs about the "auto" option [here](https://rare-technologies.com/python-lda-in-gensim-christmas-edition/). 50/*k* was chosen based on [this paper](https://doi.org/10.1073/pnas.0307752101), which is still frequently discussed.

<div class="alert alert-info">
Note: This trains 27 models for 10 epochs each using a single core (because Gensim reasons). It takes ~1 hour on my not-so-great laptop, but you can get a sense of the time frame by letting it train the first three models and print how long each took. The number of topics doesn't seem to affect training time as much as alpha does, so you really just need to see the first three iterations, which will include one model with each alpha. You can reduce the number of passes (lower value for the <tt>passes</tt> argument). If you want to work through the notebook but cannot afford to leave the notebook running for that long, you can make the following changes, but the result is that you will not log coherence or perplexity during training, and you will not be able to make some of the plots. <br>
    - remove "auto" from <tt>alphas</tt>  <br>
    - comment out the line with the "callbacks=..." argument <br>
    - change <tt>LdaModel</tt> to <tt>LdaMulticore</tt> <br>
    - add the argument <tt>workers=workers</tt> to <tt>LdaMulticore</tt> (this is defined in the first code cell of the notebook)
</div>

In [ ]:
ans = input("Start hyperparameter tuning? This takes a while. y/n ")

if ans.lower() in ["yes", "y"]:

    start_time = time.time()

    ks = [2, 3, 4, 5] + [k for k in range(10, 51, 10)]

    for k in ks:
        alphas = [[50/k]*k, "symmetric", "auto"]
        for alpha in alphas:
            inner_start_time = time.time()
            lda = LdaModel(train_corpus, id2word=dictionary, num_topics=k, passes=10, eval_every=None,
                           callbacks=[cv_coherence_logger, perplexity_logger],
                           alpha=alpha, random_state=5971) # from random.org
            if type(alpha) == list:
                lda.save(f"models/lda_k_{k}_alpha_fifty_over_k.model")
                print(f"Finished k = {k}, alpha = 50/k in {(time.time()-inner_start_time)/60:.1f} minutes.")
            else:
                lda.save(f"models/lda_k_{k}_alpha_{alpha}.model")
                print(f"Finished k = {k}, alpha = {alpha} in {(time.time()-inner_start_time)/60:.1f} minutes.")

    print(f"\nFinished in {(time.time()-start_time)/60:.1f} minutes")

### Loading the Saved Results

Now that we've trained models with different combinations of *k* and alpha, let's take a look at the results. The next cell initializes an empty dataframe before looping through each value of *k* and each value of alpha, loading the topic model we trained for each unique combination, and adding various statistics to the dataframe.

In [ ]:
metrics_df = pd.DataFrame()

# Included again in case you want to re-run the notebook without re-running the previous cell!
ks = [2, 3, 4, 5] + [k for k in range(10, 51, 10)]
alphas = ["symmetric", "auto", "fifty_over_k"]

for k in ks:
    for alpha in alphas:
        lda = LdaModel.load(f"models/lda_k_{k}_alpha_{alpha}.model")
        model_metrics = pd.DataFrame.from_dict(lda.metrics)
        model_metrics["k"] = k
        model_metrics = model_metrics.reset_index().rename(columns={"index": "Pass"}) # convert the index to a variable
        model_metrics["Pass"] = model_metrics["Pass"].apply(lambda x: x+1) # add 1, so we have 1-10 in the default setup of the notebook
        model_metrics["alpha"] = alpha
        metrics_df = pd.concat([metrics_df, model_metrics])
        
metrics_df.reset_index(inplace=True, drop=True)

In [ ]:
metrics_df.head()

In [ ]:
metrics_df.shape # 9 values of k times 3 alphas times 10 passes = 270

In [ ]:
metrics_df.to_json("data/notebook12_metrics_df.json")

In [ ]:
metrics_df = pd.read_json("data/notebook12_metrics_df.json")

### Plotting the Results

First, let's examine how things changed during the training process. We are going to create a three-by-three grid of plots for each of the metrics we recorded during training. Each cell will correspond to a specific value of *k*, will have the number of passes (epochs) on the x-axis, and will have one of our outcomes on the y-axis. In each cell, we will plot the relationship between the number of passes and the outcome for a given value of *k* for each of our three alphas.

#### Perplexity

We will start by plotting the results for perplexity over the course of the training process. The limits of the y-axis on each subplot are identical, so the values are comparable. Did the models improve with each pass through the data (epoch)?

In [ ]:
# We will keep the limits of the y-axis the same for each cell so they will be comparable
perp_y_lim = [round(metrics_df.Perplexity.min()*0.95, 1), round(metrics_df.Perplexity.max()*1.05, 1)]

ks = [2, 3, 4, 5] + [k for k in range(10, 51, 10)]

n_rows = 3
n_cols = 3
cell = 1
total = n_rows*n_cols
fig = plt.figure(figsize=(18,10))

for k in ks:
    ax = fig.add_subplot(n_rows, n_cols, cell)
    ax.set_title(f"k = {k}")
    ax.set_ylim(perp_y_lim)
    ax.set_xlabel("Epoch")
    tmp = metrics_df[metrics_df.k==k]
    for alpha in alphas:
        ax.plot("Pass", "Perplexity", data=tmp[tmp.alpha==alpha])
    if cell % 3 == 1:
        ax.set_ylabel("Perplexity")
    cell += 1
plt.tight_layout(rect=[0, 0, 1, 0.95])
fig.legend(alphas, bbox_to_anchor = (0.5, 0))
plt.suptitle("Figure 1. Held-out Perplexity during Training by k and alpha\n", fontsize=20)
plt.show()

#### Coherence

Now we will plot the results for coherence. The limits of the y-axis on each subplot are identical, so the values are comparable. Did the models improve with each pass through the data (epoch)?

In [ ]:
# We will keep the limits of the y-axis the same for each cell so they will be comparable
coh_y_lim = [round(metrics_df.Coherence.min()*0.95, 2), round(metrics_df.Coherence.max()*1.05, 2)]

n_rows = 3
n_cols = 3
cell = 1
total = n_rows*n_cols
fig = plt.figure(figsize=(18,10))

for k in ks:
    ax = fig.add_subplot(n_rows, n_cols, cell)
    ax.set_title(f"k = {k}")
    ax.set_ylim(coh_y_lim)
    ax.set_xlabel("Epoch")
    tmp = metrics_df[metrics_df.k==k]
    for alpha in alphas:
        ax.plot("Pass", "Coherence", data=tmp[tmp.alpha==alpha])
    if cell % 3 == 1:
        ax.set_ylabel("Coherence")
    cell += 1
plt.tight_layout(rect=[0, 0, 1, 0.95])
fig.legend(alphas, bbox_to_anchor = (0.5, 0))
plt.suptitle("Figure 2. Coherence during Training by k and alpha\n", fontsize=20)
plt.show()

In [ ]:
metrics_df.head()

### Choosing *k* and alpha based upon Coherence and Perplexity

Now let's turn our attention to the final model for each combination of k and alpha, meaning we will disregard performance during the early stages of training. By looking at `metrics_df[metrics_df.Pass==metrics_df.Pass.max()]`, we are looking at only the latest result for each model. We will plot *k* on the x-axis now. What kind of relationship do you see between the number of topics and the metrics we are examining? Does this differ by alpha?

In [ ]:
sns.lineplot(x="k", y="Perplexity", hue="alpha", data=metrics_df[metrics_df.Pass==metrics_df.Pass.max()])
plt.xlabel("k")
plt.ylabel("Perplexity")
plt.title("Figure 3. Held-out Perplexity by Number of Topics (k)")
plt.show()

sns.lineplot(x="k", y="Coherence", hue="alpha", data=metrics_df[metrics_df.Pass==metrics_df.Pass.max()])
plt.xlabel("k")
plt.ylabel("Coherence")
plt.title("Figure 4. Coherence by Number of Topics (k)")
plt.show()

### Computing a Different Coherence Metric for Each Saved Model

We logged one measure of coherence (<tt>"c_v"</tt>) during training, but we have the saved models. We can calculate other measures using these. Whereas the measure of coherence we used was calculated based upon the test data, we will now use [a different measure of coherence](https://aclanthology.org/D11-1024.pdf) (<tt>"u_mass"</tt>) that is calculated using the training data. (The [paper](https://svn.aksw.org/papers/2015/WSDM_Topic_Evaluation/public.pdf) that introduced the <tt>"c_v"</tt> approach found that <tt>"u_mass"</tt> was less highly correlated with human judgment.)

In [ ]:
ks = [2, 3, 4, 5] + [k for k in range(10, 51, 10)]
alphas = ["symmetric", "auto", "fifty_over_k"]

coh_umass = []

for k in ks:
    for alpha in alphas:
        lda = LdaModel.load(f"models/lda_k_{k}_alpha_{alpha}.model")
        coh = CoherenceModel(model=lda, corpus=train_corpus, dictionary=dictionary, coherence="u_mass").get_coherence()
        coh_umass += [coh]*len(set(metrics_df.Pass))
        
metrics_df["Coherence_UMass"] = coh_umass

In [ ]:
sns.lineplot(x="k", y="Coherence_UMass", hue="alpha", data=metrics_df[metrics_df.Pass==metrics_df.Pass.max()])
plt.xlabel("k")
plt.ylabel("Coherence (UMass)")
plt.title("Figure 5. A Second Measure of Coherence by Number of Topics (k)")
plt.show()

### Loading a Model to Explore

Now we can pick different valuels of *k* and alpha, load models, and explore the topics subjectively.

Run the cells below and input your answers (without quotation marks).

In [ ]:
k = input("Pick a value of k from [2, 3, 4, 5, 10, 20, 30, 40, 50]. ")
alpha = input('Pick an alpha from ["symmetric", "auto", "fifty_over_k"] ')

lda = LdaModel.load(f"models/lda_k_{k}_alpha_{alpha}.model")

dictionary = Dictionary.load("notebook12_dictionary.d")
train_corpus = pickle.load(open("notebook12_train_corpus.p", "rb"))
test_corpus = pickle.load(open("notebook12_test_corpus.p", "rb"))

## Exploring Topics via Weights and Word Clouds

In [ ]:
def display_topics(model: LdaModel):
    """Print top words and display word clouds for each topic"""
    num_topics = lda.num_topics
    topic_strings = sorted(list(lda.print_topics(lda.num_topics)), key=lambda x: x[0])
    for t, topic in enumerate(topic_strings):
        print(f"Topic {topic[0]}\n---\n{topic[1]}")
        plt.figure()
        plt.imshow(WordCloud(background_color = "white").fit_words(dict(lda.show_topic(t, 100))))
        plt.axis("off")
        plt.title("Topic " + str(t))
        plt.show()
        print()

In [ ]:
display_topics(lda)

## Exploring Topics with `pyLDAvis` (Revisited)

We will also take a closer look at `pyLDAvis`. Get a feel for it and then take a look at the blurb below about adjusting the relevance metric (the blue slider). 

In [ ]:
import pyLDAvis
import pyLDAvis.gensim_models

pyLDAvis.enable_notebook()

warnings.filterwarnings("ignore", category=DeprecationWarning)

In [ ]:
pyLDAvis.gensim_models.prepare(topic_model=lda, corpus=train_corpus, dictionary=dictionary, mds="tsne")

### Adjusting the Relevance Metric

The formula for relevance is given in footnote 2 in the bottom right of the interactive visualization.

`(term w | topic t) = λ*p(w | t) + (1-λ)*p(w | t)/p(w)`

#### Setting λ to 1

If we set λ to 1, then

`1 - λ = 1 - 1 = 0`

and the second half becomes zero: 

`(1 - λ) * p(w | t)/p(w) = 0` <br>
`(1 - 1) * p(w | t)/p(w) = 0` <br>
`      0 * p(w | t)/p(w) = 0` <br>

We are left with

`λ * p(w | t)` <br>
`1 * p(w | t)` <br>
`p(w | t)` <br>

This is simply the probability of the word given the topic.

#### Setting λ to 0

However, if we set λ to 0, then the first half becomes zero:

`λ * p(w | t) = 0` <br>
`0 * p(w | t) = 0` <br>

and we are left with

`(1 - λ) * p(w | t)/p(w)` <br>
`(1 - 0) * p(w | t)/p(w)` <br>
`1 * p(w | t)/p(w)` <br>
`p(w | t)/p(w)` <br>

This is the probability of the word given the topic, normalized by the probability of the word. This assigns more weight to words that are rarer overall, so we may see words that are more distinctive. However, these words may be rare enough that they are also not very representative of the topic.

## Exercises

<div class="alert alert-warning">
    <b>Exercise 1</b><br><br>
    When we trained the models, we used the same data, the same preprocessing choices, and the same general topic modeling algorithm with an almost identical specification. We only varied two hyperparameters–k and alpha–but got rather different results. <br><br>
    1.1 Begin by comparing two models with different numbers of topics and/or different alphas. You might pick a model with many topics and compare it to a model with few topics, or you might pick two models that are closer in number but differ in some other way. What are the values for k and alpha of each model? Why did you pick these two models?
</div>

*Your answer here*

<div class="alert alert-warning">
    1.2 The code below creates a new column, model_name, by combining the k and alpha for each model. Finish the code to display the rows of metrics_df corresponding to your chosen models in the final pass (epoch). You only need to change the variables k1, alpha1, k2, and alpha2. 
</div>

In [ ]:
metrics_df["model_name"] = metrics_df.apply(lambda row: f"{row.k}_{row.alpha}", axis=1)

# Edit k1, alpha1, k2, and alpha2. alpha1 and alpha2 should be "symmetric", "auto", or "fifty_over_k"
k1 = 0
alpha1 = ""

k2 = 0
alpha2 = ""


# Don't edit below this line
models_to_examine = [f"{k1}_{alpha1}", f"{k2}_{alpha2}"]
metrics_df[(metrics_df.model_name.isin(models_to_examine)) & (metrics_df.Pass==10)]

<div class="alert alert-warning">
    1.3 Based on these metrics (and on Figures 3 and 4), which model seems better?
</div>

*Your answer here*

<div class="alert alert-warning">
    <b>Exercise 2</b><br><br>
    2.1 Pick one model by choosing the combination of k and alpha that seem most promising to you based on some combination of the quantitative evaluations and the topics themselves. You can use the code below to examine models.
</div>

In [ ]:
k = input("Pick a value of k from [2, 3, 4, 5, 10, 20, 30, 40, 50]. ")
alpha = input('Pick an alpha from ["symmetric", "auto", "fifty_over_k"] ')

lda = LdaModel.load(f"models/lda_k_{k}_alpha_{alpha}.model")

# Load these resources in case you've restarted the notebook
dictionary = Dictionary.load("notebook12_dictionary.d")
train_corpus = pickle.load(open("notebook12_train_corpus.p", "rb"))
test_corpus = pickle.load(open("notebook12_test_corpus.p", "rb"))

# Function prints the top words and display word clouds
display_topics(lda)

pyLDAvis.gensim_models.prepare(topic_model=lda, corpus=train_corpus, dictionary=dictionary, mds="tsne")

<div class="alert alert-warning">
    2.2 Which model did you choose? Why?
</div>

*Your answer here*

<div class="alert alert-warning">
    <b>Exercise 3</b><br><br>
    3.1 Hopefully your chosen model sheds some light on various social processes discussed on r/jobs. Pick at least two topics from your chosen model. You will label them and describe how they shed light on social phenomena. <br><br>
    What would you call the first of your chosen topics? Which of the top words for the topic motivated you to choose this label?
</div>

*Your answer here*

<div class="alert alert-warning">
    3.2 What social process or processes does the topic seem to highlight?
</div>

*Your answer here*

<div class="alert alert-warning">
    3.3 What would you call the second topic you chose? Which of the top words for the topic motivated you to choose this label?
</div>

*Your answer here*

<div class="alert alert-warning">
    3.4 What social process or processes does your second chosen topic seem to highlight?
</div>

*Your answer here*

<div class="alert alert-warning">
    3.5 Does your chosen model draw attention to any issues or social processes that you might not have thought to ask about in a survey or interview? Overall, would you say the model seems useful?
</div>

*Your answer here*